In [22]:
from gruut import sentences

def english_cleaners3(text):
    """Pipeline for English text, including phonemization + punctuation"""
    punctuations = list(";:,.!?")
    phonemizer = lambda s: [
        [word.text] if word.is_major_break or word.is_minor_break else word.phonemes
        for words in sentences(s)
        for word in words
    ]

    phonemes = phonemizer(text)
    phonemes = " ".join(["".join(phn) for phn in phonemes])
    for p in punctuations:
        phonemes = phonemes.replace(f" {p}", p)
    return phonemes

In [26]:
import pandas as pd
import re
from tqdm.contrib.concurrent import process_map

df = pd.read_csv("/root/hifi-gan/en_au_dean2zak/validation.txt", sep="|", header=None, names=["id", "phonemes"])
df["phonemes"] = df["phonemes"].apply(lambda x: x.replace("_1", "").replace("_2", "").replace(". . .", "."))
# collapse multiple whitespace with regex
df["phonemes"] = df["phonemes"].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
phonemes = process_map(english_cleaners3, df["phonemes"])
df["phonemes"] = phonemes
df["id"] = df["id"].apply(lambda x: "/root/pflowtts_pytorch/en_au_dean2zak/audio/" + x + ".wav")
df.to_csv("/root/hifi-gan/en_au_dean2zak/filelists/validation.txt", sep="|", header=None, index=None)
df

/tmp/ipykernel_1637433/1896929074.py:9: TqdmWarning: Iterable length 1023 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  phonemes = process_map(english_cleaners3, df["phonemes"])


  0%|          | 0/1023 [00:00<?, ?it/s]

,id,phonemes
0,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,plˈiz tˈɛl ðˈoʊz ˈɔn ðə ˈaʊtsaɪd ðˈæt ðə pˈipə...
1,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,hˈi kˈɚlz hˈɪz hˈænd sˈoʊ ðˈæt ðə wˈɪnd dˈʌz n...
2,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,dˈaɪjə pˈʊts hˈɚ hˈænd ˈʌp.
3,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,ˈaɪ wˈɪl jˈuz ˈɪt tə ɡˈɛt tə ðˈæt bˈoʊt.
4,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,bˈʌt ʃˈi mˈeɪ dɹˈɑp ə kɹˈʌm fɚ ˈʌs tə ˈit.
...,...,...
1018,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,ðə vˈɪlɪd͡ʒɚz ɡˈɛt ðə ɡɹˈoʊs ˈɛɡz.
1019,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,ə kˈætɚpˌɪlɚ wˈɪθ mˈɔɹ ðən ə fjˈu spˈaɪnz.
1020,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,ˈɪt d͡ʒˈʌst wˈɔnts tə bˈi sˈeɪf.
1021,/root/pflowtts_pytorch/en_au_dean2zak/audio/en...,ʃˈi stˈɑɹts tə ɹˈʌn ˈænd skˈɪp bˈaɪ ðə fˈɔɹɪst.


In [1]:
from pydub import AudioSegment

def resample(path):
    sound = AudioSegment.from_file(path)
    sound.export(path.replace("wavs/", "wavs_44/"), format="wav", parameters=["-ar", "44100"])

In [5]:
from glob import glob
from tqdm.contrib.concurrent import process_map

audios = sorted(glob("/root/LJSpeech-1.1/wavs/*.wav"))
_ = process_map(resample, audios)

/tmp/ipykernel_1637433/362126750.py:5: TqdmWarning: Iterable length 13100 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  _ = process_map(resample, audios)


  0%|          | 0/13100 [00:00<?, ?it/s]

In [32]:
_pad = "_"
_punctuation = ";:,.!? "
_letters_ipa = [
    "a",
    "b",
    "d",
    "e",
    "f",
    "h",
    "i",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "p",
    "s",
    "t",
    "u",
    "v",
    "w",
    "z",
    "æ",
    "ð",
    "ŋ",
    "ɑ",
    "ɔ",
    "ə",
    "ɚ",
    "ɛ",
    "ɡ",
    "ɪ",
    "ɹ",
    "ʃ",
    "ʊ",
    "ʌ",
    "ʒ",
    "ˈ",
    "ˌ",
    "͡",
    "θ",
]

In [33]:
symbols = [_pad] + list(_punctuation) + _letters_ipa

In [34]:
len(symbols)

47